In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
df=pd.read_csv('C:/Users/jiang/Desktop/customer_behavior.csv',encoding='utf-8')
df.head()

,Unnamed: 0,cust_id,prod_id,group_id,be_type,day_id,buy_time
0,0,101,1305157,2520872,pv,2019-11-09 07:32:10,2019-11-09
1,1,101,1323287,3524611,pv,2019-11-12 01:30:35,2019-11-12
2,2,101,1338623,149293,pv,2019-11-07 17:00:14,2019-11-07
3,3,101,1341020,4690522,pv,2019-11-10 19:27:40,2019-11-10
4,4,101,1531134,2920577,pv,2019-11-07 06:02:12,2019-11-07


In [4]:
data=df[['cust_id','prod_id','group_id','be_type','day_id','buy_time']]
data1=data[(data.buy_time>='2019-11-05')&(data.buy_time<='2019-11-13')]

In [5]:
product_buy=data1.loc[data1['be_type']=='buy',['cust_id','group_id']]
product_buy_count=product_buy.groupby('group_id')['cust_id'].count().rename('销售次数')

In [7]:
product_buy_count=pd.DataFrame(product_buy_count)  

In [10]:
product_buy_count=product_buy_count.sort_values(by='销售次数',axis=0,ascending = False)

In [13]:
product_buy_count=product_buy_count.iloc[:10,:]
product_buy_count=product_buy_count.reset_index(drop = False)

In [14]:
product_buy_count.head()

,group_id,销售次数
0,1464217,3681
1,2735567,3630
2,4145914,3282
3,2885743,3165
4,4756206,3032


In [15]:
product_pv=data1.loc[data1['be_type']=='pv',['cust_id','group_id']]
product_pv_count=product_pv.groupby('group_id')['cust_id'].count().rename('点击次数')
product_pv_count=pd.DataFrame(product_pv_count)   
product_pv_count=product_pv_count.sort_values(by='点击次数',axis=0,ascending = False)
product_pv_count=product_pv_count.iloc[:10,:]
product_pv_count=product_pv_count.reset_index(drop = False)

In [16]:
product_pv_count.head()

,group_id,点击次数
0,4756206,490366
1,4145914,334181
2,2355173,328051
3,3607462,304453
4,983027,286615


In [19]:
top=pd.merge(product_pv_count,product_buy_count,how='left')
top

,group_id,点击次数,销售次数
0,4756206,490366,3032.0
1,4145914,334181,3282.0
2,2355173,328051,NaN
3,3607462,304453,NaN
4,983027,286615,2533.0
5,2520478,204187,NaN
6,4801527,196441,2725.0
7,1320394,183155,1804.0
8,2465437,159394,NaN
9,3002662,149539,1802.0


In [20]:
item_behavior=data.groupby(['group_id','be_type'])['cust_id'].count().unstack(1).rename(columns={'pv':'点击量','fav':'收藏量','cart':'加购量','buy':'购买量'}).fillna(0)
item_behavior.head()
item_behavior['转化率']=item_behavior['购买量']/item_behavior['点击量']
item_behavior.head()

be_type,购买量,加购量,收藏量,点击量,转化率
group_id,,,,,
2272,12.0,14.0,8.0,118.0,0.101695
2511,1.0,4.0,2.0,87.0,0.011494
2919,2.0,0.0,0.0,3.0,0.666667
3680,0.0,0.0,0.0,10.0,0.000000
5008,0.0,5.0,0.0,25.0,0.000000


In [21]:
item_behavior['感兴趣比率']=item_behavior['加购量']/item_behavior['点击量']
item_behavior.head()

be_type,购买量,加购量,收藏量,点击量,转化率,感兴趣比率
group_id,,,,,,
2272,12.0,14.0,8.0,118.0,0.101695,0.118644
2511,1.0,4.0,2.0,87.0,0.011494,0.045977
2919,2.0,0.0,0.0,3.0,0.666667,0.000000
3680,0.0,0.0,0.0,10.0,0.000000,0.000000
5008,0.0,5.0,0.0,25.0,0.000000,0.200000


In [22]:
item_behavior=item_behavior[item_behavior['购买量']>0]
up=item_behavior['购买量'].sum()*0.8
item_behavior=item_behavior.sort_values(by='购买量',ascending=False)
item_behavior['累计购买量']=item_behavior['购买量'].cumsum()
item_behavior['分类']=item_behavior['累计购买量'].map(lambda x:'前80%' if x<=up else '后20%')
item_behavior.head()

be_type,购买量,加购量,收藏量,点击量,转化率,感兴趣比率,累计购买量,分类
group_id,,,,,,,,
1464217,3681.0,6298.0,2803.0,72975.0,0.050442,0.086304,3681.0,前80%
2735567,3630.0,9878.0,3652.0,115233.0,0.031501,0.085722,7311.0,前80%
4145914,3282.0,18388.0,12445.0,334351.0,0.009816,0.054996,10593.0,前80%
2885743,3165.0,5780.0,3160.0,96817.0,0.032691,0.059700,13758.0,前80%
4756206,3032.0,23822.0,15818.0,490575.0,0.006181,0.048559,16790.0,前80%


In [23]:
item_behavior.groupby('分类')['分类'].count()/item_behavior['分类'].count()

分类
前80%    0.135521
后20%    0.864479
Name: 分类, dtype: float64